# Programming Best Practices and the Central Limit Theorem

This tutorial is intended to provide practice in debugging, speed optimization, and spotting bad habits in programming. The template code provided below runs, but is deeply flawed. Your goal is to improve the code following programming best practices, and along the way you'll learn something about the Central Limit Theorem.

Author: Sheila Kannappan
Last Modified: June 2021

##### Note: If this notebook becomes unresponsive, try going to the top-right where it should say "Python 3" and re-selecting the Python 3 kernel. 

In [ ]:
# ipython "magics" to enable static plot output directly to notebook and automatic reimport of modified imported code
%matplotlib inline
%load_ext autoreload
%autoreload 2

## A Little Background on Programming Best Practices

### Goals

* Easy debugging
* Easy modification
* Understandability (now, after passage of time, and to another person)
* Speed

### Strategies
* Plan – consider likely rate-limiting steps and best methodology before starting
* Modularize – test subcomponents and use branches in Git
* Check variable values, types, array sizes by hand (print statements or interrupted run)
* Don’t assume “running” = “working”; brainstorm sanity checks
* Use meaningful variable names (more than one letter!) that are not too similar
* Keep standard defaults: e.g. i, j reserved for integer loop counters
* Replace hardwired numbers with constant names at top of code or even in separate file
* Write comments (including to bookend command sets, e.g. in if-then)
* Take advantage of helpful visual appearance: white space, syntax highlighting
* Avoid loops when unnecessary (possible tradeoff with understandability)
* Manage I/O and memory; eliminate large arrays when no longer needed
* Use print and system time statements to find out where code fails/slows

### Python-specific tips
* don't define a function in the middle of a program, even though python lets you -- doing so muddles whether variables are defined locally or globally
* don't use lists when you can use numpy arrays, and especially don't append elements to dynamically build lists, unless you want glacially slow code
* pay attention to white space, it has syntactical meaning! and as a corollary, don't use tabs as white space
* standard indentation of code levels is 4 spaces (this is a matter of preference, but standardizing is essential when collaborating)
* protected code: it's generally a bad idea to import a script -- it will run at the time of import! you can use the "def main" protocol to create a "protected" code if you want your code to be both callable like a script and importable like a package

### Slight Digression: How to Write Protected Code
To understand how to write protected code, compare `templatecode.py` and `templatecodeprotected.py` in side-by-side Jupyter Lab windows (outside this notebook). For more information on writing importable modules and packages, consult [this link](https://en.wikibooks.org/wiki/Python_Programming/Modules).

## A Little Background on the Central Limit Theorem

Just for fun, the code for this tutorial will explore the Central Limit Theorem by comparing Poisson distributions with Gaussian distributions. A "distribution" is a function giving the probabilities (plotted on the y axis) associated with different outcomes (values of x), so the whole function integrates to the sum of all possible probabilities: 1.

An example of a Poisson process is counting the # of people who use the gym per hour where the count is run for a period of time nhr (set to = different times). We assume the underlying average users per hour U is fixed but the counts have "Poisson fluctuations" so N = nhr x U specifies the *expected* count (the mean of the theoretical Poisson distribution) not the observed count Nobs, whose possible values have different probabilities following a Poisson distribution with mean N. (In fact nhr may need to be very large for N to exactly equal Nobs, because Nobs is by definition an integer, but U is by definition a real number.)

Statistical theory tells us that for a Poisson process, the observed count Nobs fluctuates around the true theoretical mean N with a 68% confidence interval of +-sqrt(N) for "large N". The sleight of hand of statistics is to use the observed data to estimate N as Nobs and likewise estimate the 68% confidence interval as +-sqrt(Nobs). Thus the estimated fractional error in the count is fracerr = sqrt(Nobs)/Nobs = 1/sqrt(Nobs). The 1/sqrt explains why we get a  better estimate of U by running the count for 10 hours rather than 1 hour. However in the code we will not use data but simply compare the theoretical distributions while increasing N (or equivalently, increasing nhr).

According to the Central Limit Theorem, as we increase N, the Poisson distribution should start to look like a Gaussian. Therefore we will plot the Poisson distribution for increasing N and overplot Gaussians with the same mean N and 68% confidence interval +-sqrt(N), to see how quickly the Poisson shape approaches a Gaussian shape (i.e., to determine when we reach the "large N" limit).

## Task 1: Debugging

The cell below asks you to edit the templatecodeprotected.py script to debug it. To edit it, simply double-click on it in the list of files on the left tab, make your edits when it opens, then save it by pressing Ctrl+s or saving via the "File" tab at the top-left of the window. 
<br><br>
Your challenge: a bug is crashing the code. Try to find it using the python debugger module "pdb" as described in the tutorial [here](https://github.com/capprogram/2023bootcamp/blob/master/bestpractices/Debugging_in_Python_Python_Conquers_The_Universe.pdf) The template code is so short that pdb is not really necessary to debug it, but use pdb anyway just to get the experience for future reference. Check the size and contents of the variables at each step to determine whether they make sense. Useful commands include print, len(), np.size() and .shape(). Look at the output to see if it makes sense. Just because a code stops crashing doesn't mean it is doing what you want. Additionally, be sure that the functions you are using are fed the proper inputs. You can look up the documentation for functions from modules (such as numpy, matplotlib, etc) online.
<br><br>
Note: It is necessary to type "q" to get out of pdb before you edit and re-run the code each time.

In [ ]:
# copy templatecodeprotected.py to templatecodeprotected_mod.py so you can compare the original to your modified version
# edit templatecodeprotected_mod.py outside this notebook and re-run this cell until the bugs are gone
%run templatecodeprotected_mod

## Task 2: Speed-Up

We don't always want to optimize code speed -- sometimes it's just not important -- but you should be in the habit of avoiding silly things that slow your code down, like unnecessary loops or math operations. Measure the time taken by the whole template code as well as smaller parts of the template code and try to find inefficiencies. When you find a slow step, ask yourself whether it could be faster, and whether it matters (is it the rate-limiting step?). For now, fix it even if it's not the rate-limiting step, just for practice. Overall, you should be able to speed up this code by about a factor of 10. 

Here are two approaches to timing code:

1) You can try using the system clock via the `time` module. The times you derive with the system clock will be affected by delays from unrelated processes running in the background, but the clock is still handy for obtaining multiple timestamps to identify the bottlenecks in your code. For example, *in the templatecodeprotected_mod.py file*:

    ~~~
    import numpy as np
    import time

    init_time = time.perf_counter()  # start clock
    x = np.linspace(0,100,1000000)
    y = np.sqrt(x)

    elap_time = time.perf_counter() - init_time  # finds difference

    print("Time elapsed after linspace and sqrt commands is %0.3f ms" % (elap_time*1000))  # converts to ms
    ~~~

2) You can try using the [`%timeit`](https://ipython.org/ipython-doc/3/interactive/magics.html#magic-timeit) magic. `%timeit` is more accurate than the `time` module, but `%timeit` can only be used to time the entire code all at once. __You will need to import the protected version of the code if you want to run %timeit on it__, but you won't need to re-import because we set the %autoreload magic in the first cell of this notebook. Therefore, put the import command in a separate cell from the cell where you run %timeit repeatedly. For example, *in this notebook*:

    ~~~
    import templatecodeprotected_mod
    ~~~

    ~~~
    %timeit -n6 -r6 templatecodeprotected_mod.main()
    ~~~


In [ ]:
# Before making any changes, make a reference copy of templatecodeprotected_mod.py called "templatecodeprotected_slow.py" 
# and get a baseline time (this will vary with hardware, operating system, and what else is running).
import templatecodeslow

In [ ]:
%timeit -n6 -r6 templatecodeslow.main()

In [ ]:
# Now edit templatecodeprotected_mod.py outside this notebook and try to figure out what parts of it take the most time.
# Use the time module with print statements to check elapsed times at various points.
%run templatecodeprotected_mod

In [ ]:
# Once you think you have made the code faster, use %timeit to check your work. 
import templatecodeprotected_mod

In [ ]:
%timeit -n6 -r6 templatecodeprotected_mod.main()

### Slight Digression: timeit outside ipython
By the way, even when you're not working in ipython, you can time code using [`timeit.py`](https://docs.python.org/3/library/timeit.html). For example the following times the function "test": 

    ~~~
    def test():
        """Stupid test function"""
        L = []
        for i in range(100):
            L.append(i)

    if __name__ == '__main__':
        import timeit
        print(timeit.timeit("test()", setup="from __main__ import test"))
    ~~~

## Task 3: Clean-Up
Some things in the template code represent poor programming practice, even though they do not affect speed and are not bugs. Note examples and correct them.

In [ ]:
# copy templatecodeprotected_mod.py to templatecodeprotected_preclean.py so you can refer to the latter to see your changes
# clean up templatecodeprotected_mod.py outside this notebook and re-run this cell as you do to make sure it still works
%run templatecodeprotected_mod

## Task 4: Checking Results
Once you've got the code fixed up, you can study the plot and  to see how closely the Poisson and Gaussian distributions match each other for each value of N. If they match well, does that mean the fractional error in the observed count must be small? Explain.

Use the %load magic to insert the code directly in the cell below, then rerun the cell. Uncomment the plt.xlim command to zoom in on different parts of the plot.

In [ ]:
# %load templatecodeprotected_mod
"""
This template code is intended to provide practice in debugging, speed 
optimization, and spotting bad habits in programming. The code runs but 
is deeply flawed. If you improve the code following all the best practices 
you know, you'll learn something about both programming and the Central 
Limit Theorem.

Author: Sheila Kannappan
Last modified: September 2019
"""

# standard imports and naming conventions; uncomment as needed
import numpy as np              # basic numerical analysis
import matplotlib.pyplot as plt # plotting
import scipy.stats as stats     # statistical functions
#import pdb                      # python debugger
#import time                     # python timekeeper

def gaussfunc(xvals, mean, sigma):
    y = np.exp(-0.5*((((xvals-mean)/sigma)**2)))
    norm = 1./(sigma*np.sqrt(2. * np.pi))
    y = norm * y
    return y

def main():
    U = 8. # underlying rate of gym users per hour 
    Nct = np.array([6, 36, 216, 1296]) # typical total number of people counted (powers of 6)
    nhr = Nct/U # time to count this many people
    #labelarr = ["count for %s hr" % ihr for ihr in nhr]
    
    for i in range(0, len(Nct)):
        
        # plot probabilities of count values for range around mean
        mean = Nct[i]
        maxval = 2*mean
        xvals=np.arange(0, maxval)
        prob = stats.poisson.pmf(xvals, mean)
        plt.plot(xvals, prob, 'r', lw=3)
        plt.xlabel("count value")
        plt.ylabel("probability")
        plt.xscale("log")
        sel = np.where(np.isclose(prob,max(prob)))
        sel=sel[0][0]
        n = xvals[sel]
        probval = prob[sel]
        label = "count for %s hr" % (nhr[i])
        plt.text(n, probval, label)

        # plot Gaussian distribution with matching mean and sigma
        sigma=np.sqrt(mean)
        y = gaussfunc(xvals, mean, sigma)
        plt.plot(xvals, y, 'b')
        #plt.xlim([,])
    
if __name__ == '__main__':
    main()
    plt.show()
